In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re
import string
import nltk
from nltk.corpus import stopwords

from wordcloud import WordCloud, STOPWORDS

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

import pattern
from pattern.en import lemma, lexeme

In [ ]:
df=pd.read_csv('amazon_alexa.tsv', sep='\t')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['review_length']=df['verified_reviews'].apply(len)
df.head()

In [ ]:
df.groupby('review_length').describe().sample(10)


In [ ]:
df.groupby('rating').describe()


In [ ]:
ratings=df['rating'].value_counts()

import plotly.express as px
fig = px.funnel_area(names=ratings.index,
                    values=ratings.values,
                    title='Distribution of Alexa Ratings')
fig.show()

In [ ]:
feedback=df['feedback'].value_counts()
fig = px.pie(df, values=feedback.values, names=feedback.index,
            title='Distribution of Feedback')
fig.show()

In [ ]:
reviews=px.histogram(df,x="review_length",title='Distribution of Length of Reviews',labels={"review_length":"Review Length"})
reviews.show()


# Data Cleaning

In [ ]:
# REMOVE 3 STAR REVIEWS AND CATEGORIZE ABOVE AND BELOW 3
df = df[df['rating']!=3]
df['new_rating']=np.where(df['rating']>=4, 'positive', 'negative')
df.head()

In [ ]:
# REMOVE PUNCTUATION AND MAKE ALL LOWERCASE
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['new_reviews'] = df['verified_reviews'].map(alphanumeric).map(punc_lower)
df.head()

In [ ]:
# REMOVE STOP WORDS
stop = set(stopwords.words('english'))

stop_words = ['-PRON-','’s','be','"','“','’','u','—','','get','would' ]

def remove_stops(comment):
    stop_free = [i for i in comment.split() if i not in stop]
    got_stop_free = " ".join([i for i in stop_free if i not in stop_words])
    return got_stop_free

df['new_reviews']= df['new_reviews'].apply(remove_stops)

In [ ]:
df.head()

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])   

alexa_df['reviews']= alexa_df['reviews'].apply(space)